In [1]:
import os

train_action_path = os.path.join('train_action_history.csv')
train_cdna_path   = os.path.join('train_cdna_data.csv')
test_action_path  = os.path.join('test_action_history.csv')
test_cdna_path    = os.path.join('test_cdna_data.csv')
test_customers_path = os.path.join('test_customers.csv')


In [2]:
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

import logging

In [3]:
# 1) Load train_action_history
train_action = pd.read_csv(
    train_action_path,
    parse_dates=['send_timestamp', 'open_timestamp'],  # parse timestamps
    # low_memory=True can help if memory is limited
)

# 2) Load train_cdna_data
train_cdna = pd.read_csv(train_cdna_path)  # parse_dates if you have a cDNA snapshot date column

# 3) Similarly for test datasets
test_action = pd.read_csv(
    test_action_path,
    parse_dates=['send_timestamp', 'open_timestamp'],
)
test_cdna = pd.read_csv(test_cdna_path)
test_customers = pd.read_csv(test_customers_path)


C:\Users\Ishaan\AppData\Local\Temp\ipykernel_22148\584636974.py:9: DtypeWarning: Columns (7,12,13,15,16,17,18,19,29,37,45,46,47,56,58,66,69,71,74,228) have mixed types. Specify dtype option on import or set low_memory=False.
  train_cdna = pd.read_csv(train_cdna_path)  # parse_dates if you have a cDNA snapshot date column
C:\Users\Ishaan\AppData\Local\Temp\ipykernel_22148\584636974.py:16: DtypeWarning: Columns (15,16,17,18,19,45,46,47,56,58,71,228) have mixed types. Specify dtype option on import or set low_memory=False.
  test_cdna = pd.read_csv(test_cdna_path)


In [4]:
# Check if (customer_code, batch_id, offer_subid) might have duplicates
duplicates = train_action.duplicated(subset=['customer_code', 'batch_id', 'Offer_subid'])
print("Number of exact duplicates in train_action:", duplicates.sum())

# If any duplicates exist, drop them
if duplicates.sum() > 0:
    train_action = train_action.drop_duplicates(subset=['customer_code', 'batch_id', 'Offer_subid'])


Number of exact duplicates in train_action: 0


In [5]:
# Ensure send_timestamp is always <= open_timestamp (if open_timestamp is not null)
invalid_mask = (train_action['open_timestamp'].notna()) & (train_action['send_timestamp'] > train_action['open_timestamp'])
print("Invalid (send > open) rows:", invalid_mask.sum())

# Potentially drop or fix these rows if needed
train_action = train_action[~invalid_mask]


Invalid (send > open) rows: 0


In [6]:
train_action['is_open'] = train_action['open_timestamp'].notnull().astype(int)
test_action['is_open'] = test_action['open_timestamp'].notnull().astype(int)


In [7]:
# Ensure 'send_timestamp' is in datetime format
train_action['send_timestamp'] = pd.to_datetime(train_action['send_timestamp'], errors='coerce')

# Now you can extract the day_of_week and hour
train_action['day_of_week'] = train_action['send_timestamp'].dt.weekday  # Monday=0, Sunday=6
train_action['hour'] = train_action['send_timestamp'].dt.hour


# Ensure 'send_timestamp' is in datetime format
test_action['send_timestamp'] = pd.to_datetime(test_action['send_timestamp'], errors='coerce')

# Now you can extract the day_of_week and hour
test_action['day_of_week'] = test_action['send_timestamp'].dt.weekday  # Monday=0, Sunday=6
test_action['hour'] = test_action['send_timestamp'].dt.hour

In [8]:
def get_slot_number_for_send(timestamp):
    """
    Returns slot_1 to slot_28 based on:
      - day_of_week: 0..6 (Mon..Sun)
      - hour: from 9..21 in 3-hour increments
    Returns None if outside 9:00-21:00
    """
    day = timestamp.weekday()  # 0=Mon, 6=Sun
    hour = timestamp.hour

    # If hour is outside [9..20], it doesn't map to a valid 9AM-9PM slot
    # (21 is 9PM boundary, so hour=21 is outside).
    if hour < 9 or hour >= 21:
        return None

    # Within 9..20, figure out which 3-hour block:
    # 9-11 -> block 0
    # 12-14 -> block 1
    # 15-17 -> block 2
    # 18-20 -> block 3
    block = (hour - 9) // 3  # integer division

    # Each day has 4 blocks, so overall slot = day*4 + block + 1
    slot_index = day*4 + block + 1
    return f"slot_{slot_index}"

train_action['send_slot'] = train_action['send_timestamp'].apply(get_slot_number_for_send)
test_action['send_slot'] = test_action['send_timestamp'].apply(get_slot_number_for_send)


In [9]:
def get_slot_number_for_open(ts):
    if pd.isna(ts):
        return None
    day = ts.weekday()
    hour = ts.hour
    if hour < 9 or hour >= 21:
        return None
    block = (hour - 9) // 3
    slot_index = day * 4 + block + 1
    return f"slot_{slot_index}"

train_action['open_slot'] = train_action['open_timestamp'].apply(get_slot_number_for_open)
test_action['open_slot'] = test_action['open_timestamp'].apply(get_slot_number_for_open)


In [10]:
def get_slot_number_for_send_ML(ts):
    if pd.isna(ts):
        return 0
    day = ts.weekday()
    hour = ts.hour
    if hour < 9 or hour >= 21:
        return 0
    block = (hour - 9) // 3
    slot_index = day * 4 + block + 1
    return int(slot_index)

train_action['send_slot_ML'] = train_action['send_timestamp'].apply(get_slot_number_for_send_ML)
test_action['send_slot_ML'] = test_action['send_timestamp'].apply(get_slot_number_for_send_ML)


In [11]:
def get_slot_number_for_open_ML(ts):
    if pd.isna(ts):
        return 0
    day = ts.weekday()
    hour = ts.hour
    if hour < 9 or hour >= 21:
        return 0
    block = (hour - 9) // 3
    slot_index = day * 4 + block + 1
    return int(slot_index)

train_action['open_slot_ML'] = train_action['open_timestamp'].apply(get_slot_number_for_open_ML)
test_action['open_slot_ML'] = test_action['open_timestamp'].apply(get_slot_number_for_open_ML)


In [12]:
# Customer-level engagement metrics
train_action['open_delay'] = (train_action['open_timestamp'] - train_action['send_timestamp']).dt.total_seconds()

customer_aggs = train_action.groupby('customer_code').agg(
    open_rate=('is_open', 'mean'),
    total_opens=('is_open', 'sum'),
    total_emails=('is_open', 'count'),
    unique_slots=('send_slot_ML', 'nunique'),
    most_common_send=('send_slot_ML', lambda x: x.mode().iloc[0] if not x.empty else 0),
    most_common_open=('open_slot_ML', lambda x: x[x > 0].mode().iloc[0] if any(x > 0) else 0),
    avg_response_time=('open_delay', lambda x: x[x > 0].mean() if any(x > 0) else 0),
    med_response_time=('open_delay', lambda x: x[x > 0].median() if any(x > 0) else 0)
).reset_index()

# Time-based success patterns
hourly_stats = train_action.groupby('hour').agg(
    hourly_count=('is_open', 'count'),
    hourly_mean=('is_open', 'mean'),
    hourly_sum=('is_open', 'sum')
).reset_index()

daily_stats = train_action.groupby('day_of_week').agg(
    daily_count=('is_open', 'count'),
    daily_mean=('is_open', 'mean'),
    daily_sum=('is_open', 'sum')
).reset_index()

slot_stats = train_action.groupby('send_slot_ML').agg(
    slot_count=('is_open', 'count'),
    slot_mean=('is_open', 'mean'),
    slot_sum=('is_open', 'sum')
).reset_index()

# Repeat for test data
test_action['open_delay'] = (test_action['open_timestamp'] - test_action['send_timestamp']).dt.total_seconds()

test_customer_aggs = test_action.groupby('customer_code').agg(
    open_rate=('is_open', 'mean'),
    total_opens=('is_open', 'sum'),
    total_emails=('is_open', 'count'),
    unique_slots=('send_slot_ML', 'nunique'),
    most_common_send=('send_slot_ML', lambda x: x.mode().iloc[0] if not x.empty else 0),
    most_common_open=('open_slot_ML', lambda x: x[x > 0].mode().iloc[0] if any(x > 0) else 0),
    avg_response_time=('open_delay', lambda x: x[x > 0].mean() if any(x > 0) else 0),
    med_response_time=('open_delay', lambda x: x[x > 0].median() if any(x > 0) else 0)
).reset_index()

In [13]:
train_action.head()

,customer_code,Offer_id,Offer_subid,batch_id,product_category,product_sub_category,send_timestamp,open_timestamp,is_open,day_of_week,hour,send_slot,open_slot,send_slot_ML,open_slot_ML,open_delay
0,00199d3467a7191db5bfa4e5f9a62eeb96fb0b602c3ec5...,AC_100044882,VR_200098111,70000078,CC_ACQ_SECURED,SECURED_ACQ,2024-10-22 17:35:27+00:00,NaT,0,1,17,slot_7,None,7,0,NaN
1,001f2abab1bccc25d00bba68fea57a81cab1c76d485515...,AC_100048426,VR_200108485,10000253,SIP / MF,ACQUISITION,2024-08-09 18:20:18+00:00,NaT,0,4,18,slot_20,None,20,0,NaN
2,00298fc11fb6924004c041f141f92c3c74e209d9a737d9...,AC_100052603,VR_200127708,70000074,CC_INORGANIC,EMI,2024-09-20 10:45:20+00:00,2024-09-20 10:45:31+00:00,1,4,10,slot_17,slot_17,17,17,11.0
3,0038a46221c0175fc8938ebc8aef8d0f83b3ac1ad84662...,AC_100047006,VR_200105035,70000072,CC_ACQ_SECURED,SECURED_ACQ,2024-09-06 12:37:16+00:00,NaT,0,4,12,slot_18,None,18,0,NaN
4,00409395a831af7fd41d0ec70a5be3bda13d64cb3e4be9...,AC_100046122,VR_200100845,10000246,RURAL,BALANCE BUILD UP,2024-06-23 11:03:13+00:00,NaT,0,6,11,slot_25,None,25,0,NaN


In [14]:
train_cdna.head()

,CUSTOMER_CODE,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v294,v295,v296,v297,v298,v299,v300,v301,v302,batch_date
0,ab617a6a0a8582f4aaa1aeda38fd73377cb911e6096a98...,50-54,NaN,NaN,99,THANE,99999,NaN,INDIA,IN,...,187,44,204,150,73,271,164,170,246,2024-09-27
1,6e8e3227297409f3f33578400302825263cadc2ed0d1a0...,35-39,NaN,NaN,ZZ,Pune,ZZ,NaN,INDIA,411,...,187,44,204,150,73,271,164,170,246,2024-09-27
2,1b42f270aba756b64d7ae4e2409313097b0c91f7c2f2c7...,20-24,NaN,NaN,99,BHIWANI,95013,NaN,INDIA,IN,...,187,44,204,150,73,271,164,170,246,2024-09-27
3,06a4aae9b531a518260c7d0d88811cc202fd0d3e46d9ea...,20-24,NaN,NaN,NaN,CHIKMAGALUR,NaN,NaN,INDIA,NaN,...,187,44,204,150,73,271,164,170,246,2024-09-27
4,0e4fa92b7a41dc019c9f40457e180e94ca60d0b5c7128e...,30-34,NaN,NaN,NaN,NASIK,NaN,NaN,INDIA,NaN,...,187,44,204,150,73,271,164,170,246,2024-09-27


In [15]:
# Let's analyze the distribution of null ratios
null_ratios = train_cdna.isnull().mean()
print("Distribution of null ratios:")
print(null_ratios.value_counts(bins=10).sort_index())

# Count columns at different thresholds
thresholds = [0.25, 0.5, 0.75, 0.9]
for t in thresholds:
    cols = null_ratios[null_ratios <= t].index
    print(f"\nColumns with <= {t*100}% nulls: {len(cols)}")

Distribution of null ratios:
(-0.002, 0.1]     62
(0.1, 0.2]         5
(0.2, 0.3]         7
(0.3, 0.4]        32
(0.4, 0.5]         3
(0.5, 0.6]        37
(0.6, 0.7]        14
(0.7, 0.8]        15
(0.8, 0.9]        16
(0.9, 1.0]       112
Name: count, dtype: int64

Columns with <= 25.0% nulls: 68

Columns with <= 50.0% nulls: 109

Columns with <= 75.0% nulls: 171

Columns with <= 90.0% nulls: 191


In [16]:
# Select columns with <=25% nulls
null_ratios = train_cdna.isnull().mean()
cols_to_keep = null_ratios[null_ratios <= 0.25].index.tolist()

# Let's analyze the data types of these columns
dtypes_analysis = train_cdna[cols_to_keep].dtypes.value_counts()
print("\nData types in selected columns:")
print(dtypes_analysis)


Data types in selected columns:
int64      35
object     23
bool        9
float64     1
Name: count, dtype: int64


In [17]:
# Check object columns content
object_cols = train_cdna.select_dtypes(include=['object']).columns
for col in object_cols:
    unique_vals = train_cdna[col].nunique()
    sample_vals = train_cdna[col].dropna().head(3).tolist()
    print(f"\nColumn: {col}")
    print(f"Unique values: {unique_vals}")
    print(f"Sample values: {sample_vals}")

object_cols_test = test_cdna.select_dtypes(include=['object']).columns
for col in object_cols:
    unique_vals = test_cdna[col].nunique()
    sample_vals = test_cdna[col].dropna().head(3).tolist()
    print(f"\nColumn: {col}")
    print(f"Unique values: {unique_vals}")
    print(f"Sample values: {sample_vals}")


Column: CUSTOMER_CODE
Unique values: 220699
Sample values: ['ab617a6a0a8582f4aaa1aeda38fd73377cb911e6096a984784be6f123024ea50', '6e8e3227297409f3f33578400302825263cadc2ed0d1a06e9b9d0f50d94ba216', '1b42f270aba756b64d7ae4e2409313097b0c91f7c2f2c7aa8b06e89c46fc754f']

Column: v2
Unique values: 22
Sample values: ['50-54', '35-39', '20-24']

Column: v5
Unique values: 25
Sample values: ['99', 'ZZ', '99']

Column: v6
Unique values: 2743
Sample values: ['THANE', 'Pune', 'BHIWANI']

Column: v7
Unique values: 208
Sample values: ['99999', 'ZZ', '95013']

Column: v8
Unique values: 1
Sample values: ['xxxxx', 'xxxxx', 'xxxxx']

Column: v9
Unique values: 105
Sample values: ['INDIA', 'INDIA', 'INDIA']

Column: v10
Unique values: 210
Sample values: ['IN', '411', 'IN']

Column: v11
Unique values: 82454
Sample values: ['2018-08-25T00:00:00.000Z', '2018-08-03T14:12:16.000Z', '2021-10-08T00:00:00.000Z']

Column: v13
Unique values: 1
Sample values: ['G', 'G', 'G']

Column: v14
Unique values: 1
Sample values

In [18]:
class CustomImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.strategies = {}
        self.fill_values = {}
        self.age_medians = None
        self.city_tiers = None

    def fit(self, X, y=None):
        # Age imputation
        mask_train = X['v2'].notna() & X['v80'].notna()
        df_train_valid = X.loc[mask_train, ['v2', 'v80']]
        self.age_medians = df_train_valid.groupby('v2')['v80'].median()
        self.overall_median_v80 = X['v80'].median(skipna=True)

        # Normalize and impute cities/tiers
        X['v6'] = X['v6'].str.upper()
        mask = X['v6'].notna() & X['v101'].notna()
        df_valid = X.loc[mask, ['v6', 'v101']]
        mode_df = df_valid.groupby('v6')['v101'].agg(lambda x: x.value_counts().index[0])
        self.city_tiers = dict(zip(mode_df.index, mode_df.values))
        self.fallback_tier = X['v101'].dropna().mode()[0]

        # Normalize gender
        X['v54'] = X['v54'].apply(lambda x: 'MALE' if str(x).upper() in ['M', 'MALE'] else 'FEMALE' if str(x).upper() in ['F', 'FEMALE'] else x)

        # Normalize country
        X['v9'] = X['v9'].apply(lambda x: 'INDIA' if str(x).upper() in ['IN', 'IN - INDIA', 'INDIA'] else x)

        # Regular imputation
        self.columns_to_drop = ['v2', 'v27', 'v29', 'v31']
        X_filtered = X.drop(columns=self.columns_to_drop, errors='ignore')
        
        for column in X_filtered.columns:
            if column not in ['v80', 'v101']:
                if X_filtered[column].dtype == 'object':
                    if X_filtered[column].nunique() > 100:
                        self.strategies[column] = 'constant'
                        self.fill_values[column] = 'MISSING'
                    else:
                        self.strategies[column] = 'mode'
                        self.fill_values[column] = X_filtered[column].mode()[0]
                elif X_filtered[column].dtype == 'bool':
                    self.strategies[column] = 'mode'
                    self.fill_values[column] = X_filtered[column].mode()[0]
                elif X_filtered[column].dtype in ['int64', 'float64']:
                    if X_filtered[column].nunique() > 20:
                        self.strategies[column] = 'median'
                        self.fill_values[column] = X_filtered[column].median()
                    else:
                        self.strategies[column] = 'mode'
                        self.fill_values[column] = X_filtered[column].mode()[0]
        return self

    def transform(self, X):
        X_copy = X.copy()
        
        # Age imputation using v2 before dropping it
        X_copy['v80'] = X_copy.apply(lambda row: self.age_medians[row['v2']] if pd.notna(row['v2']) and row['v2'] in self.age_medians.index else self.overall_median_v80 if pd.isna(row['v80']) else row['v80'], axis=1)
        X_copy['v80'] = X_copy['v80'].astype(int)

        # Normalize and impute
        X_copy['v6'] = X_copy['v6'].str.upper()
        X_copy['v54'] = X_copy['v54'].apply(lambda x: 'MALE' if str(x).upper() in ['M', 'MALE'] else 'FEMALE' if str(x).upper() in ['F', 'FEMALE'] else x)
        X_copy['v9'] = X_copy['v9'].apply(lambda x: 'INDIA' if str(x).upper() in ['IN', 'IN - INDIA', 'INDIA'] else x)
        
        # City-tier imputation
        X_copy['v101'] = X_copy.apply(lambda row: self.city_tiers.get(row['v6'], self.fallback_tier) if pd.isna(row['v101']) else row['v101'], axis=1)

        # Drop columns after using them for imputation
        X_copy = X_copy.drop(columns=self.columns_to_drop, errors='ignore')

        # Regular imputation
        for column in X_copy.columns:
            if column not in ['v80', 'v101']:
                X_copy[column].fillna(self.fill_values.get(column, 'MISSING'), inplace=True)

        return X_copy

In [19]:
# First, let's keep only columns with <=50% nulls
null_ratios = train_cdna.isnull().mean()
cols_to_keep = null_ratios[null_ratios <= 0.25].index.tolist()

# Filter columns
train_cdna_filtered = train_cdna[cols_to_keep].copy()
test_cdna_filtered = test_cdna[cols_to_keep].copy()

# Initialize and fit the imputer
imputer = CustomImputer()
imputer = imputer.fit(train_cdna_filtered)

# Transform both train and test data
train_cdna_imputed = imputer.transform(train_cdna_filtered)
test_cdna_imputed = imputer.transform(test_cdna_filtered)

# Verify no nulls remain
print("Nulls in train after imputation:", train_cdna_imputed.isnull().sum().sum())
print("Nulls in test after imputation:", test_cdna_imputed.isnull().sum().sum())

# Show shape of data before and after
print("\nShape of data:")
print(f"Train CDNA - Original: {train_cdna.shape}, After imputation: {train_cdna_imputed.shape}")
print(f"Test CDNA - Original: {test_cdna.shape}, After imputation: {test_cdna_imputed.shape}")

# Show sample of imputed data
print("\nSample of imputed data:")
print(train_cdna_imputed.head())

C:\Users\Ishaan\AppData\Local\Temp\ipykernel_22148\2356013800.py:75: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_copy[column].fillna(self.fill_values.get(column, 'MISSING'), inplace=True)
C:\Users\Ishaan\AppData\Local\Temp\ipykernel_22148\2356013800.py:75: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_cop

Nulls in train after imputation: 0
Nulls in test after imputation: 0

Shape of data:
Train CDNA - Original: (1285402, 303), After imputation: (1285402, 64)
Test CDNA - Original: (68450, 303), After imputation: (68450, 64)

Sample of imputed data:
                                       CUSTOMER_CODE           v6     v9  \
0  ab617a6a0a8582f4aaa1aeda38fd73377cb911e6096a98...        THANE  INDIA   
1  6e8e3227297409f3f33578400302825263cadc2ed0d1a0...         PUNE  INDIA   
2  1b42f270aba756b64d7ae4e2409313097b0c91f7c2f2c7...      BHIWANI  INDIA   
3  06a4aae9b531a518260c7d0d88811cc202fd0d3e46d9ea...  CHIKMAGALUR  INDIA   
4  0e4fa92b7a41dc019c9f40457e180e94ca60d0b5c7128e...        NASIK  INDIA   

       v10                       v11 v30  \
0       IN  2018-08-25T00:00:00.000Z  11   
1      411  2018-08-03T14:12:16.000Z  15   
2       IN  2021-10-08T00:00:00.000Z  95   
3  MISSING  2024-03-18T00:00:00.000Z  01   
4  MISSING  2024-02-26T00:00:00.000Z  04   

                               

In [20]:
# Fix column names first
train_action['customer_code'] = train_action['customer_code']
train_cdna_imputed['CUSTOMER_CODE'] = train_cdna_imputed['CUSTOMER_CODE']
test_cdna_imputed['CUSTOMER_CODE'] = test_cdna_imputed['CUSTOMER_CODE']

# Train join
train_cdna_imputed['batch_date'] = pd.to_datetime(train_cdna_imputed['batch_date'])
train_action['send_date'] = pd.to_datetime(train_action['send_timestamp'].dt.date)

# Process train data in chunks
chunk_size = 100000  # Adjust based on your RAM
train_merged_chunks = []

for start in range(0, len(train_action), chunk_size):
    end = start + chunk_size
    chunk = train_action.iloc[start:end].copy()
    chunk['send_date'] = pd.to_datetime(chunk['send_timestamp'].dt.date)
    
    chunk_merged = pd.merge_asof(
        chunk.sort_values('send_date'),
        train_cdna_imputed.sort_values('batch_date'),
        left_on='send_date',
        right_on='batch_date',
        left_by='customer_code',
        right_by='CUSTOMER_CODE',
        direction='backward'
    )
    train_merged_chunks.append(chunk_merged)
    
    # Free memory
    del chunk_merged
    del chunk
    
train_merged = pd.concat(train_merged_chunks)
del train_merged_chunks

# Test data (usually smaller, can process at once)
test_merged = test_action.merge(
    test_cdna_imputed,
    left_on='customer_code',
    right_on='CUSTOMER_CODE',
    how='left'
)

print(f"Train merged shape: {train_merged.shape}")
print(f"Test merged shape: {test_merged.shape}")

# Validation checks
def validate_merges():
    print("Train Validation:")
    print(f"Original shapes - Action: {train_action.shape}, CDNA: {train_cdna_imputed.shape}")
    print(f"Merged shape: {train_merged.shape}")
    print(f"Missing values in merged: {train_merged.isnull().sum().sum()}")
    print(f"Unique customers in action: {train_action['customer_code'].nunique()}")
    print(f"Unique customers in merged: {train_merged['customer_code'].nunique()}")
    
    print("\nTest Validation:")
    print(f"Original shapes - Action: {test_action.shape}, CDNA: {test_cdna_imputed.shape}")
    print(f"Merged shape: {test_merged.shape}")
    print(f"Missing values in merged: {test_merged.isnull().sum().sum()}")
    print(f"Unique customers in action: {test_action['customer_code'].nunique()}")
    print(f"Unique customers in merged: {test_merged['customer_code'].nunique()}")

validate_merges()

Train merged shape: (8797911, 81)
Test merged shape: (1456503, 80)
Train Validation:
Original shapes - Action: (8797911, 17), CDNA: (1285402, 64)
Merged shape: (8797911, 81)
Missing values in merged: 20923384
Unique customers in action: 221204
Unique customers in merged: 221204

Test Validation:
Original shapes - Action: (1456503, 16), CDNA: (68450, 64)
Merged shape: (1456503, 80)
Missing values in merged: 2523063
Unique customers in action: 64700
Unique customers in merged: 64700


In [21]:
# Save in chunks to manage memory
def save_in_chunks(df, filename, chunk_size=100000):
    # First chunk with headers
    df.iloc[:chunk_size].to_csv(filename, index=False)
    
    # Append remaining chunks without headers
    for i in range(chunk_size, len(df), chunk_size):
        df.iloc[i:i+chunk_size].to_csv(filename, mode='a', header=False, index=False)

# Save train and test
save_in_chunks(train_merged, 'train_merged.csv')
save_in_chunks(test_merged, 'test_merged.csv')


In [22]:
# List all variables in memory
for obj in globals():
   if isinstance(globals()[obj], pd.DataFrame):
       print(f"DataFrame '{obj}': {globals()[obj].shape}, Memory usage: {globals()[obj].memory_usage().sum() / 1024**2:.2f} MB")


RuntimeError: dictionary changed size during iteration

In [ ]:
del test_cdna_filtered
del train_cdna_filtered
del train_cdna_imputed
del test_cdna_imputed
del train_cdna
del test_cdna
del train_action
del test_action

In [ ]:
# Check missing values by column
missing_by_col = train_merged.isnull().sum()
missing_pct_by_col = (train_merged.isnull().sum() / len(train_merged)) * 100

# Create summary DataFrame
missing_summary = pd.DataFrame({
   'Missing Values': missing_by_col,
   'Missing %': missing_pct_by_col
})

# Sort by most missing
missing_summary = missing_summary[missing_summary['Missing Values'] > 0].sort_values('Missing Values', ascending=False)

print("Train Missing Values Summary:")
print(missing_summary)

# Same for test
test_missing_by_col = test_merged.isnull().sum()
test_missing_pct = (test_merged.isnull().sum() / len(test_merged)) * 100

test_missing_summary = pd.DataFrame({
   'Missing Values': test_missing_by_col,
   'Missing %': test_missing_pct
})

test_missing_summary = test_missing_summary[test_missing_summary['Missing Values'] > 0].sort_values('Missing Values', ascending=False)

print("\nTest Missing Values Summary:")
print(test_missing_summary)

# NEURAL NETWORK MODEL

- After this point, the 2 dataframes were downloaded as csvs, and uploaded again as dataframes due to recurring BSODs due to the large size of the dataframes. Currently, the code is continuing to use dataframes already present, however it has not been checked. Please run on your own discretion.

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# Create engagement metrics
def add_engagement_metrics(df):
   aggs = df.groupby('customer_code').agg(
       open_rate=('is_open', 'mean'),
       total_opens=('is_open', 'sum'),
       total_emails=('is_open', 'count')
   ).reset_index()
   return df.merge(aggs, on='customer_code', how='left')

train_merged = add_engagement_metrics(train_merged)
test_merged = add_engagement_metrics(test_merged)

In [ ]:
def optimize_dataframe(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
        elif df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
        elif df[col].dtype == 'object':
            if df[col].nunique() / len(df[col]) < 0.5:  # If low cardinality
                df[col] = df[col].astype('category')
    return df

train_merged = optimize_dataframe(pd.read_csv('train_merged.csv'))
test_merged = optimize_dataframe(pd.read_csv('test_merged.csv'))

In [ ]:
test_customers = pd.read_csv('test_customers.csv')

In [ ]:
# Define feature columns
CATEGORICAL_COLS = [
    'product_category', 'product_sub_category', 'send_slot', 
    'v6', 'v9', 'v10', 'v34', 'v35', 'v36', 'v37', 'v54', 'v101', 'v102'
]

NUMERICAL_COLS = [
    'is_open', 'day_of_week', 'hour', 'send_slot_ML',
    'v80', 'v287', 'v288', 'v289', 'v290', 'v291', 'v292', 
    'v293', 'v294', 'v295', 'v296', 'v297', 'v298', 'v299', 
    'v300', 'v301', 'v302'
]


In [ ]:
def prepare_features(df, encoders=None, scaler=None, is_training=True):
    df = df.copy()
    
    if is_training:
        encoders = {col: LabelEncoder() for col in CATEGORICAL_COLS}
        scaler = StandardScaler()
    
    for col in CATEGORICAL_COLS:
        if df[col].dtype.name == 'category':
            df[col] = df[col].astype(str)
        df[col] = df[col].fillna('MISSING')
        if is_training:
            # Add 'MISSING' to training data before encoding
            unique_vals = set(df[col].unique())
            unique_vals.add('MISSING')
            df[col] = df[col].astype(str)
            encoders[col].fit(list(unique_vals))
            df[col] = encoders[col].transform(df[col])
        else:
            known_categories = set(encoders[col].classes_)
            df[col] = df[col].astype(str).apply(lambda x: x if x in known_categories else 'MISSING')
            df[col] = encoders[col].transform(df[col])
    
    for col in NUMERICAL_COLS:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    
    if is_training:
        df[NUMERICAL_COLS] = scaler.fit_transform(df[NUMERICAL_COLS])
    else:
        df[NUMERICAL_COLS] = scaler.transform(df[NUMERICAL_COLS])
    
    feature_cols = CATEGORICAL_COLS + NUMERICAL_COLS
    X = df[feature_cols]
    
    if is_training:
        return X, encoders, scaler
    return X
    

In [ ]:
def build_model(input_dim, num_slots=28):
    """
    Neural network architecture optimized for slot prediction:
    
    Architecture:
    - Input layer matching feature dimensions
    - 3 Dense layers with decreasing sizes to create information bottleneck
    - BatchNormalization after each Dense layer to stabilize learning
    - Dropout layers with decreasing rates to prevent overfitting
    - L2 regularization on first layer to control weight magnitudes
    
    Parameters:
    - input_dim: Number of input features
    - num_slots: Number of output slots (default 28)
    
    Key Design Choices:
    1. Layer sizes (256->128->64): Creates gradual information bottleneck
    2. Dropout rates (0.5->0.4->0.3): Higher dropout early, lower later
    3. L2 regularization: Prevents any single feature from dominating
    4. Learning rate 0.0003: Conservative to prevent overshooting
    """
    model = Sequential([
        Dense(256, activation='relu', kernel_regularizer=l2(0.01), input_dim=input_dim),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.4),
        
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        
        Dense(num_slots, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.0003),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
def validate_predictions(predictions, slot_rankings):
    """
    Validates prediction outputs for correctness and format
    
    Checks:
    1. Correct number of slots (28)
    2. No duplicate slots
    3. Valid slot format (slot_X where X is 1-28)
    4. Probability distribution sums to ~1
    """
    for customer_predictions in slot_rankings:
        if len(customer_predictions) != 28:
            raise ValueError(f"Each prediction must have 28 slots, got {len(customer_predictions)}")
        
        if len(set(customer_predictions)) != 28:
            raise ValueError("Predictions contain duplicate slots")
        
        # Validate slot format
        for slot in customer_predictions:
            if not slot.startswith("slot_"):
                raise ValueError(f"Invalid slot format: {slot}")
            try:
                slot_num = int(slot.split('_')[1])
                if not 1 <= slot_num <= 28:
                    raise ValueError(f"Invalid slot number: {slot_num}")
            except (IndexError, ValueError):
                raise ValueError(f"Invalid slot format: {slot}")
    
    # Validate probabilities
    if isinstance(predictions, np.ndarray):
        prob_sums = np.sum(predictions, axis=1)
        if not np.allclose(prob_sums, 1.0, atol=1e-6):
            raise ValueError("Prediction probabilities do not sum to 1")

In [ ]:
def evaluate_model(model, X_val, y_val, history):
    """
    Comprehensive model evaluation with multiple metrics
    
    Metrics:
    1. Accuracy (overall slot prediction accuracy)
    2. Precision, Recall, F1 (per slot performance)
    3. Training history analysis
    4. Confusion patterns
    """
    # Prediction metrics
    predictions = model.predict(X_val)
    y_pred = predictions.argmax(axis=1)
    y_true = y_val.argmax(axis=1)
    
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    
    # Training history analysis
    final_train_loss = history.history['loss'][-1]
    final_val_loss = history.history['val_loss'][-1]
    loss_diff = final_val_loss - final_train_loss
    
    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'final_train_loss': final_train_loss,
        'final_val_loss': final_val_loss,
        'loss_difference': loss_diff
    }
    
    logger.info("Model Evaluation Results:")
    for metric, value in metrics.items():
        logger.info(f"{metric}: {value:.4f}")
        
    return metrics

In [ ]:
def get_default_slots(slot_rankings):
    if not slot_rankings:  # If no rankings yet, return sequential slots
        return [f"slot_{i+1}" for i in range(28)]
        
    all_rankings = np.array([r for r in slot_rankings if len(r) == 28])
    slot_freq = np.zeros((28, 28))
    for ranking in all_rankings:
        for pos, slot in enumerate(ranking):
            slot_num = int(slot.split('_')[1]) - 1
            slot_freq[pos, slot_num] += 1
    
    default_slots = []
    used_slots = set()
    for pos in range(28):
        available_slots = [(i, slot_freq[pos, i]) for i in range(28) if i not in used_slots]
        best_slot = max(available_slots, key=lambda x: x[1])[0]
        used_slots.add(best_slot)
        default_slots.append(f"slot_{best_slot+1}")
    return default_slots


In [ ]:
def train_and_predict(train_data, test_data, customer_codes):
    # Prepare training data
    X_train, encoders, scaler = prepare_features(train_data, is_training=True)
    
    # Create target variable (one-hot encoded slots)
    # Shift slots down by 1 to handle slot_0
    y_slots = train_data['send_slot_ML'].apply(lambda x: x - 1 if x > 0 else 27)
    y_train = pd.get_dummies(y_slots, columns=range(28)).values
    
    # Split training data
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42
    )
    
    # Build and train model
    model = build_model(X_train.shape[1])
    
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=1,
        batch_size=1024,
        callbacks=[early_stopping],
        verbose=1
    )
    
    # Prepare test data
    test_data = test_data[test_data['customer_code'].isin(test_customers['CUSTOMER_CODE'])]
    X_test = prepare_features(test_data, encoders, scaler, is_training=False)
    
    # Get predictions
    pred_probs = model.predict(X_test)
    
    # Average predictions per customer
    customer_preds = {}
    for i, customer in enumerate(test_data['customer_code']):
        if customer not in customer_preds:
            customer_preds[customer] = pred_probs[i]
        else:
            customer_preds[customer] += pred_probs[i]
    
    # Convert averaged predictions to rankings
    slot_rankings = []
    final_customers = []
    
    # Initialize default_slots with sequential order
    default_slots = [f"slot_{i+1}" for i in range(28)]
    
    for customer_code in customer_codes:
        if customer_code in customer_preds:
            probs = customer_preds[customer_code]
            ranked_slots = np.argsort(probs)[::-1]
            slot_names = [f"slot_{i+1}" for i in ranked_slots]
            slot_rankings.append(slot_names)
            
            if len(slot_rankings) % 1000 == 0:  # Update default slots periodically
                default_slots = get_default_slots(slot_rankings)
        else:
            slot_rankings.append(default_slots)
        final_customers.append(customer_code)

    # Evaluate model
    metrics = evaluate_model(model, X_val, y_val, history)
    
    # Get predictions
    pred_probs = model.predict(X_test)
    
    # Validate predictions before creating rankings
    validate_predictions(pred_probs, slot_rankings)
    
    # Create submission DataFrame
    submission = pd.DataFrame({
        'customer_code': final_customers,
        'predicted_slots_order': slot_rankings
    })
    
    return submission, history, model


In [ ]:
# 1. Train and predict
submission, history, model = train_and_predict(
    train_merged, test_merged, test_customers['CUSTOMER_CODE']
)

# 2. Save submission
submission.to_csv('submission.csv', index=False)